In [50]:
# Import libraries and dependencies
import numpy as np
import pandas as pd
from pathlib import Path
import hvplot
import hvplot.pandas
from IPython.display import Markdown

pd.set_option("display.max_rows", 2000)
pd.set_option("display.max_columns", 2000)
pd.set_option("display.width", 1000)

In [51]:
# Read Signal CSV as a Dataframe

# Set the file path
filepath = Path("../Resources/signals_fouryear.csv")

# Read the CSV located at the file path into a Pandas DataFrame
signals_fouryear_df = pd.read_csv(filepath, parse_dates=True, infer_datetime_format=True)

# Print the DataFrame
signals_fouryear_df.head()

# Read Closing Price CSV as a Dataframe

# Set the file path
filepath = Path("../Resources/stockprice_fouryear.csv")

# Read the CSV located at the file path into a Pandas DataFrame
stockprice_fouryear_df = pd.read_csv(filepath, parse_dates=True, infer_datetime_format=True)

# Print the DataFrame
stockprice_fouryear_df.head()


,date,TSLA,QQQ,earnings flag
0,2017-01-03,43.397999,115.847992,0
1,2017-01-04,45.397999,116.477921,0
2,2017-01-05,45.349998,117.136932,0
3,2017-01-06,45.801998,118.164192,0
4,2017-01-09,46.256001,118.551819,0


### 20 & 50-Day Moving Averages: Testing Buy/Sell Algorithmic Trading Signals

In [53]:
# Grab just the `date` and `TSLA` from the closing price dataset
signals_df = stockprice_fouryear_df.loc[:, ['date', 'TSLA']].copy()

# Set the `date` column as the index
signals_df = signals_df.set_index("date", drop=True)

# Set the short window and long windows
short_window = 20
long_window = 50

# Generate the short and long moving averages (20 and 50 days, respectively)
signals_df['SMA20'] = signals_df['TSLA'].rolling(window=short_window).mean()
signals_df['SMA50'] = signals_df['TSLA'].rolling(window=long_window).mean()
signals_df['Signal'] = 0.0

# Generate the trading signal 0 or 1,
# where 0 is when the SMA20 is under the SMA50, and
# where 1 is when the SMA20 is higher (or crosses over) the SMA50
signals_df['Signal'][short_window:] = np.where(
    signals_df['SMA20'][short_window:] > signals_df['SMA50'][short_window:], 1.0, 0.0
)

# Calculate the points in time at which a position should be taken, 1 or -1
signals_df['Entry/Exit'] = signals_df['Signal'].diff()

# Print the DataFrame
signals_df.tail()

,TSLA,SMA20,SMA50,Signal,Entry/Exit
date,,,,,
2020-12-24,661.770020,623.862003,513.927201,1.0,0.0
2020-12-28,663.690002,627.758502,518.223401,1.0,0.0
2020-12-29,665.989990,632.678003,522.749800,1.0,0.0
2020-12-30,694.780029,638.179004,528.028801,1.0,0.0
2020-12-31,705.669983,645.021503,533.703401,1.0,0.0


In [54]:
# Visualize exit position relative to close price
exit = signals_df[signals_df['Entry/Exit'] == -1.0]['TSLA'].hvplot.scatter(
    color='red',
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize entry position relative to close price
entry = signals_df[signals_df['Entry/Exit'] == 1.0]['TSLA'].hvplot.scatter(
    color='green',
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = signals_df[['TSLA']].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = signals_df[['SMA20', 'SMA50']].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

# Overlay plots
entry_exit_plot = security_close * moving_avgs * entry * exit
entry_exit_plot.opts(xaxis=None)

:Overlay
   .Curve.I         :Curve   [date]   (TSLA)
   .NdOverlay.I     :NdOverlay   [Variable]
      :Curve   [date]   (value)
   .Scatter.TSLA.I  :Scatter   [date]   (TSLA)
   .Scatter.TSLA.II :Scatter   [date]   (TSLA)

In [55]:
# Set initial capital
initial_capital = float(1000000)

# Set the share size
share_size = 3000

# Take a 3000 share position where the dual moving average crossover is 1 (SMA20 is greater than SMA50)
signals_df['Position'] = share_size * signals_df['Signal']

# Find the points in time where a 3000 share position is bought or sold
signals_df['Entry/Exit Position'] = signals_df['Position'].diff()

# Multiply share price by entry/exit positions and get the cumulatively sum
signals_df['Portfolio Holdings'] = signals_df['TSLA'] * signals_df['Entry/Exit Position'].cumsum()

# Subtract the initial capital by the portfolio holdings to get the amount of liquid cash in the portfolio
signals_df['Portfolio Cash'] = initial_capital - (signals_df['TSLA'] * signals_df['Entry/Exit Position']).cumsum()

# Get the total portfolio value by adding the cash amount by the portfolio holdings (or investments)
signals_df['Portfolio Total'] = signals_df['Portfolio Cash'] + signals_df['Portfolio Holdings']

# Calculate the portfolio daily returns
signals_df['Portfolio Daily Returns'] = signals_df['Portfolio Total'].pct_change()

# Calculate the cumulative returns
signals_df['Portfolio Cumulative Returns'] = (1 + signals_df['Portfolio Daily Returns']).cumprod() - 1

# Print the DataFrame
signals_df.tail(10)

,TSLA,SMA20,SMA50,Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Daily Returns,Portfolio Cumulative Returns
date,,,,,,,,,,,,
2020-12-17,655.900024,591.220000,492.271600,1.0,0.0,3000.0,0.0,1.967700e+06,269086.044312,2.236786e+06,0.046501,1.236786
2020-12-18,695.000000,601.006500,497.653200,1.0,0.0,3000.0,0.0,2.085000e+06,269086.044312,2.354086e+06,0.052441,1.354086
2020-12-21,649.859985,609.019000,501.970400,1.0,0.0,3000.0,0.0,1.949580e+06,269086.044312,2.218666e+06,-0.057526,1.218666
2020-12-22,640.340027,614.943503,505.931201,1.0,0.0,3000.0,0.0,1.921020e+06,269086.044312,2.190106e+06,-0.012873,1.190106
2020-12-23,645.979980,619.473502,509.917800,1.0,0.0,3000.0,0.0,1.937940e+06,269086.044312,2.207026e+06,0.007726,1.207026
2020-12-24,661.770020,623.862003,513.927201,1.0,0.0,3000.0,0.0,1.985310e+06,269086.044312,2.254396e+06,0.021463,1.254396
2020-12-28,663.690002,627.758502,518.223401,1.0,0.0,3000.0,0.0,1.991070e+06,269086.044312,2.260156e+06,0.002555,1.260156
2020-12-29,665.989990,632.678003,522.749800,1.0,0.0,3000.0,0.0,1.997970e+06,269086.044312,2.267056e+06,0.003053,1.267056
2020-12-30,694.780029,638.179004,528.028801,1.0,0.0,3000.0,0.0,2.084340e+06,269086.044312,2.353426e+06,0.038098,1.353426


In [56]:
# Visualize exit position relative to total portfolio value
exit = signals_df[signals_df['Entry/Exit'] == -1.0]['Portfolio Total'].hvplot.scatter(
    color='red',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize entry position relative to total portfolio value
entry = signals_df[signals_df['Entry/Exit'] == 1.0]['Portfolio Total'].hvplot.scatter(
    color='green',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize total portoflio value for the investment
total_portfolio_value = signals_df[['Portfolio Total']].hvplot(
    line_color='lightgray',
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Overlay plots
portfolio_entry_exit_plot = total_portfolio_value * entry * exit
portfolio_entry_exit_plot.opts(xaxis=None)

:Overlay
   .Curve.I                    :Curve   [date]   (Portfolio Total)
   .Scatter.Portfolio_Total.I  :Scatter   [date]   (Portfolio Total)
   .Scatter.Portfolio_Total.II :Scatter   [date]   (Portfolio Total)

In [57]:
# Prepare DataFrame for metrics
metrics = [
    'Annual Return',
    'Cumulative Returns',
    'Annual Volatility',
    'Sharpe Ratio',
    'Sortino Ratio']

columns = ['Backtest']

# Initialize the DataFrame with index set to evaluation metrics and column as `Backtest` (just like PyFolio)
portfolio_evaluation_df = pd.DataFrame(index=metrics, columns=columns)
portfolio_evaluation_df

# Calculate cumulative return
portfolio_evaluation_df.loc['Cumulative Returns'] = signals_df['Portfolio Cumulative Returns'][-1]

# Calculate annualized return
portfolio_evaluation_df.loc['Annual Return'] = (
    signals_df['Portfolio Daily Returns'].mean() * 252
)

# Calculate annual volatility
portfolio_evaluation_df.loc['Annual Volatility'] = (
    signals_df['Portfolio Daily Returns'].std() * np.sqrt(252)
)

# Calculate Sharpe Ratio
portfolio_evaluation_df.loc['Sharpe Ratio'] = (
    signals_df['Portfolio Daily Returns'].mean() * 252) / (
    signals_df['Portfolio Daily Returns'].std() * np.sqrt(252)
)

# Calculate Downside Return
sortino_ratio_df = signals_df[['Portfolio Daily Returns']].copy()
sortino_ratio_df.loc[:,'Downside Returns'] = 0

target = 0
mask = sortino_ratio_df['Portfolio Daily Returns'] < target
sortino_ratio_df.loc[mask, 'Downside Returns'] = sortino_ratio_df['Portfolio Daily Returns']**2
portfolio_evaluation_df

# Calculate Sortino Ratio
down_stdev = np.sqrt(sortino_ratio_df['Downside Returns'].mean()) * np.sqrt(252)
expected_return = sortino_ratio_df['Portfolio Daily Returns'].mean() * 252
sortino_ratio = expected_return/down_stdev

portfolio_evaluation_df.loc['Sortino Ratio'] = sortino_ratio
portfolio_evaluation_df.head()

,Backtest
Annual Return,0.242616
Cumulative Returns,1.3861
Annual Volatility,0.22102
Sharpe Ratio,1.09771
Sortino Ratio,1.69492


In [58]:
# Initialize trade evaluation DataFrame with columns
trade_evaluation_df = pd.DataFrame(
    columns=[
        'Stock', 
        'Entry Date', 
        'Exit Date', 
        'Shares', 
        'Entry Share Price', 
        'Exit Share Price', 
        'Entry Portfolio Holding', 
        'Exit Portfolio Holding', 
        'Profit/Loss']
)

trade_evaluation_df

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss


In [59]:
# Initialize iterative variables
entry_date = ''
exit_date = ''
entry_portfolio_holding = 0
exit_portfolio_holding = 0
share_size = 0
entry_share_price = 0
exit_share_price = 0

# Loop through signal DataFrame
# If `Entry/Exit` is 1, set entry trade metrics
# Else if `Entry/Exit` is -1, set exit trade metrics and calculate profit,
# Else if index is 2020-12-31, close open position to calculate final metric set.
# Then append the record to the trade evaluation DataFrame
for index, row in signals_df.iterrows():
    if row['Entry/Exit'] == 1:
        entry_date = index
        entry_portfolio_holding = abs(row['Portfolio Holdings'])
        share_size = row['Entry/Exit Position']
        entry_share_price = row['TSLA']

    elif row['Entry/Exit'] == -1:
        exit_date = index
        exit_portfolio_holding = abs(row['TSLA'] * row['Entry/Exit Position'])
        exit_share_price = row['TSLA']
        profit_loss = exit_portfolio_holding - entry_portfolio_holding
        trade_evaluation_df = trade_evaluation_df.append(
            {
                'Stock': 'TSLA',
                'Entry Date': entry_date,
                'Exit Date': exit_date,
                'Shares': share_size,
                'Entry Share Price': entry_share_price,
                'Exit Share Price': exit_share_price,
                'Entry Portfolio Holding': entry_portfolio_holding,
                'Exit Portfolio Holding': exit_portfolio_holding,
                'Profit/Loss': profit_loss
            },
            ignore_index=True)
    elif index == "2020-12-31":
        exit_date = index
        exit_portfolio_holding = abs(row['TSLA'] * row['Position'])
        exit_share_price = row['TSLA']
        profit_loss = exit_portfolio_holding - entry_portfolio_holding
        trade_evaluation_df = trade_evaluation_df.append(
            {
                'Stock': 'TSLA',
                'Entry Date': entry_date,
                'Exit Date': exit_date,
                'Shares': share_size,
                'Entry Share Price': entry_share_price,
                'Exit Share Price': exit_share_price,
                'Entry Portfolio Holding': entry_portfolio_holding,
                'Exit Portfolio Holding': exit_portfolio_holding,
                'Profit/Loss': profit_loss
            },
            ignore_index=True)

# Print the DataFrame
trade_evaluation_df

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss
0,TSLA,2017-03-15,2017-03-21,3000.0,51.146000,50.136002,1.534380e+05,1.504080e+05,-3029.994965
1,TSLA,2017-03-31,2017-07-21,3000.0,55.660000,65.680000,1.669800e+05,1.970400e+05,30060.001373
2,TSLA,2017-08-23,2017-10-13,3000.0,70.554001,71.113998,2.116620e+05,2.133420e+05,1679.992676
3,TSLA,2017-12-26,2018-03-05,3000.0,63.458000,66.669998,1.903740e+05,2.000100e+05,9635.993958
4,TSLA,2018-03-13,2018-03-16,3000.0,68.367996,64.269997,2.051040e+05,1.928100e+05,-12293.998718
5,TSLA,2018-06-04,2018-06-05,3000.0,59.348000,58.226002,1.780440e+05,1.746780e+05,-3365.993500
6,TSLA,2018-06-06,2018-06-08,3000.0,63.900002,63.532001,1.917000e+05,1.905960e+05,-1104.000092
7,TSLA,2018-06-12,2018-07-27,3000.0,68.554001,59.436001,2.056620e+05,1.783080e+05,-27354.000092
8,TSLA,2018-08-22,2018-09-06,3000.0,64.328003,56.189999,1.929840e+05,1.685700e+05,-24414.012909
9,TSLA,2018-11-05,2019-01-04,3000.0,68.279999,63.537998,2.048400e+05,1.906140e+05,-14226.001740


In [60]:
price_df = signals_df[['TSLA', 'SMA20', 'SMA50']]
price_chart = price_df.hvplot.line()
price_chart.opts(xaxis=None)

:NdOverlay   [Variable]
   :Curve   [date]   (value)

In [61]:
portfolio_evaluation_df.reset_index(inplace=True)
portfolio_evaluation_table = portfolio_evaluation_df.hvplot.table()
portfolio_evaluation_table

:Table   [index,Backtest]

In [62]:
trade_evaluation_table = trade_evaluation_df.hvplot.table()
trade_evaluation_table

:Table   [Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss]

In [73]:
# Assemble dashboard visualization
display(Markdown("# Trading Dashboard"))
display(Markdown("### Signaled by 20 & 50-Day Moving Averages"))
display(price_chart)
portfolio_evaluation_table + trade_evaluation_table

# Trading Dashboard

### Signaled by 20 & 50-Day Moving Averages

:NdOverlay   [Variable]
   :Curve   [date]   (value)

:Layout
   .Table.I  :Table   [index,Backtest]
   .Table.II :Table   [Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss]

In [74]:
# Total P&L for 20 & 50-Day Moving Average Algorithm
profit_2050 = trade_evaluation_df["Profit/Loss"].sum()
dollar_profit_2050 = '${:,.2f}'.format(profit_2050)
print(f"The algorithm that utilizes the 20 and 50-Day Moving Averages, returned a profit of: {dollar_profit_2050}")

The algorithm that utilizes the 20 and 50-Day Moving Averages, returned a profit of: $1,386,095.99
